In [ ]:
import cv2
import numpy as np

def nothing(x):
    pass

# Load image
image = frames[166]

# Create a window
cv2.namedWindow('image')

# Create trackbars for color change
# Hue is from 0-179 for Opencv
cv2.createTrackbar('HMin', 'image', 0, 179, nothing)
cv2.createTrackbar('SMin', 'image', 0, 255, nothing)
cv2.createTrackbar('VMin', 'image', 0, 255, nothing)
cv2.createTrackbar('HMax', 'image', 0, 179, nothing)
cv2.createTrackbar('SMax', 'image', 0, 255, nothing)
cv2.createTrackbar('VMax', 'image', 0, 255, nothing)

# Set default value for Max HSV trackbars
cv2.setTrackbarPos('HMax', 'image', 179)
cv2.setTrackbarPos('SMax', 'image', 255)
cv2.setTrackbarPos('VMax', 'image', 255)

# Initialize HSV min/max values
hMin = sMin = vMin = hMax = sMax = vMax = 0
phMin = psMin = pvMin = phMax = psMax = pvMax = 0

while(1):
    # Get current positions of all trackbars
    hMin = cv2.getTrackbarPos('HMin', 'image')
    sMin = cv2.getTrackbarPos('SMin', 'image')
    vMin = cv2.getTrackbarPos('VMin', 'image')
    hMax = cv2.getTrackbarPos('HMax', 'image')
    sMax = cv2.getTrackbarPos('SMax', 'image')
    vMax = cv2.getTrackbarPos('VMax', 'image')

    # Set minimum and maximum HSV values to display
    lower = np.array([hMin, sMin, vMin])
    upper = np.array([hMax, sMax, vMax])

    # Convert to HSV format and color threshold
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    result = cv2.bitwise_and(image, image, mask=mask)

    # Print if there is a change in HSV value
    if((phMin != hMin) | (psMin != sMin) | (pvMin != vMin) | (phMax != hMax) | (psMax != sMax) | (pvMax != vMax) ):
        print("(hMin = %d , sMin = %d, vMin = %d), (hMax = %d , sMax = %d, vMax = %d)" % (hMin , sMin , vMin, hMax, sMax , vMax))
        phMin = hMin
        psMin = sMin
        pvMin = vMin
        phMax = hMax
        psMax = sMax
        pvMax = vMax

    # Display result image
    cv2.imshow('image', result)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()

In [ ]:
print(tesserocr.tesseract_version())  # print tesseract-ocr version
print(tesserocr.get_languages())  # prints tessdata path and list of available languages

In [ ]:
cnt = 0
skipXFrames = 20
while cnt < 25:
    while i < len(res):
        entry = res[i]
        if len(entry) == 0:
            i+=1
            continue
        if entry['Type'] == 'IN_GAME':
            for side in ['left', 'right']:
                cropped = Image.fromarray(frames[i]).crop(getSubImageBox(BOUNDING_BOXES['IN_GAME']['Score'], side))
                cv2.imwrite(f'../tests/qualityData/score/{side}/score_{side}_{cnt}.jpg', np.array(cropped))
            cnt += 1
            i += skipXFrames
        if cnt >= 25:
            break
        i += 1


In [ ]:
import os
import glob

class TestRunner(object):
    QUALITY_TESTS = os.path.join('..', 'tests', 'qualityData')

    def __init__(self):
        pass

    def _getTestNumber(self,fileName:str) -> int:
        return int(fileName.split('_')[-1].split('.')[0])

    def _getBaseFolderPath(self, moment:str, testCase:str, side:str) -> str:
        return os.path.join(TestRunner.QUALITY_TESTS, testCase.lower(), side)
    
    def _getCorrectResults(self, moment:str, testCase:str, side:str) -> list:
        with open(os.path.join(self._getBaseFolderPath(moment, testCase, side), 'labels.txt')) as file:
            lines = {idx:line.rstrip() for idx,line in enumerate(file)}
        return lines

    def _convertToCorrectType(self, value:str, testCase:str):
        if testCase.lower().startswith('score'):
            return int(value)
        return value

    def getTestResults(self, moment:str, testCase:str, side:str, considerBelowXConfidenceAsFailing:float=70, debug:bool=False):
        imgs = {}
        totalTestCases = 0
        correct = 0
        confidences = []
        answers = self._getCorrectResults(moment, testCase, side)
        for filename in glob.glob(os.path.join(self._getBaseFolderPath(moment, testCase, side),'*.jpg')):
            im=Image.open(filename)
            testNo = self._getTestNumber(filename)
            try:
                parsedVal, confidences = parseFromImage(im, BOUNDING_BOXES[moment][testCase], side, debug=debug)
            except:
                parsedVal, confidences = '', [0]
            totalTestCases += 1
            confidence = np.min(confidences)
            if parsedVal == self._convertToCorrectType(answers[testNo], testCase) and confidence >= considerBelowXConfidenceAsFailing:
                correct += 1
            elif debug:
                display(f'Failed to properly parse the following image / had low confidence | testNo={testNo}')
                display(f'Correct value `{answers[testNo]}`, parsed value `{parsedVal}`, passing confidence was {considerBelowXConfidenceAsFailing}, actual confidence was {confidence}')
                display(im)
            confidences.append(confidence)
        print(f'Final results for {testCase} | {side} ')
        print(f'% correct = {100.0*correct/totalTestCases}% ({correct} out of {totalTestCases})')
        print(f'Confidence (p0, p25, p50, p75): {np.percentile(confidences, [0, 25, 50, 75])}')
            

In [ ]:
tests = TestRunner()
tests.getTestResults('IN_GAME', 'Score', 'left', debug=True)

In [ ]:
im.resize((int(im.size[0]*1.5), int(im.size[1])), resample=Image.NEAREST)

In [ ]:
def imageTransformationForScore(im:Image) -> Image:
    tmp = im
    tmp = cv2.cvtColor(np.array(tmp), cv2.COLOR_BGR2GRAY)
    (th, newimg) = cv2.threshold(tmp, 215, 255, cv2.THRESH_BINARY)
    #lower = np.array([0,0,196])
    #upper = np.array([179,39,255])
    #mask = cv2.inRange(tmp, lower, upper)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4,4))
    opening = cv2.morphologyEx(newimg, cv2.MORPH_OPEN, kernel)
    kernel = np.ones((1,1),np.uint8)
    erosion = cv2.erode(opening,kernel,iterations = 5)
    return Image.fromarray(255-erosion)

im = Image.open(r"C:\wd\MSBL-Parser\tests\qualityData\in_game\score\left\score_left_10.jpg")
im = imageTransformation(im, 'Score')[0]
im.save("2.jpg")
display(im)

for val in [tesserocr.PSM.OSD_ONLY, tesserocr.PSM.AUTO_OSD, tesserocr.PSM.AUTO_ONLY, tesserocr.PSM.AUTO, tesserocr.PSM.SINGLE_COLUMN, tesserocr.PSM.SINGLE_BLOCK_VERT_TEXT, tesserocr.PSM.SINGLE_BLOCK, tesserocr.PSM.SINGLE_LINE, tesserocr.PSM.SINGLE_WORD, tesserocr.PSM.CIRCLE_WORD, tesserocr.PSM.SINGLE_CHAR, tesserocr.PSM.RAW_LINE]:
    display(f'PSM option: {val}')
    display(parseImage(im,  numbersOnly=True, tesserocrOptions={'psm': val, 'oem':1}))

In [ ]:
import sys
sys.path.append("..") # almost every `.py` need this

from src.helpers import KeyItemDetails, ParsingConfig, GamePhaseDetail, KeyItemDetail, Coordinates, GamePhaseDetails
from src.constants import FileType, GamePhase, GameSide, KeyItem, RESOLUTION
from src.parsing import parse, getFramesFromFileContentUsingParsingConfig

gamePhaseDetails = GamePhaseDetails([
    GamePhaseDetail(
        gamePhase=GamePhase.IN_GAME,
        keyItemDetails= KeyItemDetails([
            KeyItemDetail(
                Coordinates(875,0,1040,65),
                KeyItem.TIME,
                GameSide.NONE,
                numbersOnly=False,
                tesserocrOptions={}
            ),
            KeyItemDetail(
                Coordinates(185,85,400,125),
                KeyItem.TEAM_NAME,
                GameSide.LEFT,
                numbersOnly=False,
                tesserocrOptions={}
            ),
            KeyItemDetail(
                Coordinates(185,85,400,125).flipAlongYAxis(),
                KeyItem.TEAM_NAME,
                GameSide.RIGHT,
                numbersOnly=False,
                tesserocrOptions={}
            ),
            KeyItemDetail(
                Coordinates(700,20,825,84),
                KeyItem.SCORE,
                GameSide.LEFT,
                numbersOnly=True,
                tesserocrOptions={}
            ),
            KeyItemDetail(
                Coordinates(700,20,825,84).flipAlongYAxis(),
                KeyItem.SCORE,
                GameSide.RIGHT,
                numbersOnly=True,
                tesserocrOptions={}
            ),
        ]),
        identifyingKeyItem=(KeyItem.TIME, GameSide.NONE)
    ),
    GamePhaseDetail(
        gamePhase=GamePhase.GOAL_SCORED_LEFT_HAND_SIDE,
        keyItemDetails= KeyItemDetails([
            KeyItemDetail(
                Coordinates(250,850,380,900),
                KeyItem.TIME,
                GameSide.NONE,
                numbersOnly=False,
                tesserocrOptions={}
            ),
            KeyItemDetail(
                Coordinates(1100,735,1330,900),
                KeyItem.SCORE,
                GameSide.LEFT,
                numbersOnly=True,
                tesserocrOptions={}
            ),
            KeyItemDetail(
                Coordinates(1450,735,1680,900),
                KeyItem.SCORE,
                GameSide.RIGHT,
                numbersOnly=True,
                tesserocrOptions={}
            ),
            KeyItemDetail(
                Coordinates(490,850,900,900),
                KeyItem.TEAM_NAME,
                GameSide.LEFT,
                numbersOnly=False,
                tesserocrOptions={}
            ),
            KeyItemDetail(
                Coordinates(250,925,1000,1000),
                KeyItem.FLAVOR_TEXT,
                GameSide.NONE,
                numbersOnly=True,
                tesserocrOptions={}
            )
        ]),
        identifyingKeyItem=(KeyItem.TIME, GameSide.NONE)
    ),
    GamePhaseDetail(
        gamePhase=GamePhase.GOAL_SCORED_RIGHT_HAND_SIDE,
        keyItemDetails= KeyItemDetails([
            KeyItemDetail(
                Coordinates(250,850,380,900).flipAlongYAxis(),
                KeyItem.TIME,
                GameSide.NONE,
                numbersOnly=False,
                tesserocrOptions={}
            ),
            KeyItemDetail(
                Coordinates(1100,735,1330,900).flipAlongYAxis(),
                KeyItem.SCORE,
                GameSide.LEFT,
                numbersOnly=True,
                tesserocrOptions={}
            ),
            KeyItemDetail(
                Coordinates(1450,735,1680,900).flipAlongYAxis(),
                KeyItem.SCORE,
                GameSide.RIGHT,
                numbersOnly=True,
                tesserocrOptions={}
            ),
            KeyItemDetail(
                Coordinates(490,850,900,900).flipAlongYAxis(),
                KeyItem.TEAM_NAME,
                GameSide.RIGHT,
                numbersOnly=False,
                tesserocrOptions={}
            ),
            KeyItemDetail(
                Coordinates(250,925,1000,1000).flipAlongYAxis(),
                KeyItem.FLAVOR_TEXT,
                GameSide.NONE,
                numbersOnly=True,
                tesserocrOptions={}
            )
        ]),
        identifyingKeyItem=(KeyItem.TIME, GameSide.NONE)
    ),
    GamePhaseDetail(
        gamePhase=GamePhase.IN_GAME_FINAL_RESULT_LEFT_HAND_SIDE,
        keyItemDetails= KeyItemDetails([
            KeyItemDetail(
                Coordinates(1100,735,1330,900),
                KeyItem.SCORE,
                GameSide.LEFT,
                numbersOnly=True,
                tesserocrOptions={}
            ),
            KeyItemDetail(
                Coordinates(330,850,700,900),
                KeyItem.TEAM_NAME,
                GameSide.LEFT,
                numbersOnly=False,
                tesserocrOptions={}
            ),
            KeyItemDetail(
                Coordinates(1450,735,1680,900),
                KeyItem.SCORE,
                GameSide.RIGHT,
                numbersOnly=True,
                tesserocrOptions={}
            ),
            KeyItemDetail(
                Coordinates(250,925,1000,1000),
                KeyItem.FLAVOR_TEXT,
                GameSide.NONE,
                numbersOnly=True,
                tesserocrOptions={}
            )
        ]),
        identifyingKeyItem=(KeyItem.SCORE, GameSide.LEFT)
    ),
    GamePhaseDetail(
        gamePhase=GamePhase.IN_GAME_FINAL_RESULT_RIGHT_HAND_SIDE,
        keyItemDetails= KeyItemDetails([
                    KeyItemDetail(
                        Coordinates(1100,735,1330,900).flipAlongYAxis(),
                        KeyItem.SCORE,
                        GameSide.LEFT,
                        numbersOnly=True,
                        tesserocrOptions={}
                    ),
                    KeyItemDetail(
                        Coordinates(1450,735,1680,900).flipAlongYAxis(),
                        KeyItem.SCORE,
                        GameSide.RIGHT,
                        numbersOnly=True,
                        tesserocrOptions={}
                    ),
                KeyItemDetail(
                    Coordinates(330,850,700,900).flipAlongYAxis(),
                    KeyItem.TEAM_NAME,
                    GameSide.RIGHT,
                    numbersOnly=False,
                    tesserocrOptions={}
                ),
                KeyItemDetail(
                    Coordinates(250,925,1000,1000).flipAlongYAxis(),
                    KeyItem.FLAVOR_TEXT,
                    GameSide.NONE,
                    numbersOnly=True,
                    tesserocrOptions={}
                )
            
    ]),
        identifyingKeyItem=(KeyItem.SCORE, GameSide.LEFT)
    )
])



parseConfig = ParsingConfig(\
    videoOrImageToParsePath         =  r'C:\Users\hkuser\Videos\2023-03-17 09-25-05.mkv',\
    FileType                        = FileType.VIDEO,\
    gamePhaseDetails                = gamePhaseDetails,\
    imageCacheFolderPath            = r"..\src\data",\
    resolution                      = RESOLUTION,\
    processEveryXSecondsFromVideo   = 2,\
    fpsOfInputVideo                 = 60,\
    outputFolder                    = None,\
    debug                           = False,\
    ignoreParsingErrors             = False,\
)

result = parse(parseConfig)

In [ ]:
[(idx, frame.gamePhase, frame.get(KeyItem.TIME, GameSide.NONE),frame.get(KeyItem.SCORE, GameSide.RIGHT)) for idx, frame in enumerate(result.parsedFrames)]

In [ ]:
result.parsedFrames[5].getValue(KeyItem.TIME, GameSide.NONE)

In [ ]:
for i in range(170,180):
    display(result.parsedFrames[i].image)

In [ ]:
result.keyTimes

In [ ]:
result.dumpAllUnparseableImages('../src/data')

In [ ]:
from src.parsing import ImageStore
import os
store = ImageStore(os.path.join('..', 'src', 'data'), parseConfig.gamePhaseDetails, debug=True)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from src.parsing import imageTransformation
%matplotlib inline

firstValue=True
for game,keyTimes in result.keyTimes.items():
    if firstValue:
        firstValue=False
        continue
    prevscoreLeft = 0
    prevscoreRight = 0
    prevTime = None
    inGame = keyTimes[GamePhase.IN_GAME]
    for i in range(len(inGame)):
        idx = inGame[i]
        scoreLeft = result.parsedFrames[idx].getValue(KeyItem.SCORE, GameSide.LEFT)
        scoreRight = result.parsedFrames[idx].getValue(KeyItem.SCORE, GameSide.RIGHT)
        time = 240-result.parsedFrames[idx].getValue(KeyItem.TIME, GameSide.NONE).total_seconds()
        if scoreLeft is None or scoreRight is None:
            continue
        if scoreLeft < prevscoreLeft or scoreRight < prevscoreRight:
            print(f'time =`{time}`, scoreLeft=`{scoreLeft}, scoreRight=`{scoreRight}`')
            print(f'prevTime =`{prevTime}`, prevscoreLeft=`{prevscoreLeft}, prevscoreRight=`{prevscoreRight}`')
            if i >0:
                display(result.parsedFrames[inGame[i-1]].image)
                scoreLeftIm = result.parsedFrames[inGame[i-1]].image.crop(parseConfig.gamePhaseDetails[0].keyItemDetails[-1].coords.box())
                display(scoreLeftIm)  
                scoreRightIm = result.parsedFrames[inGame[i-1]].image.crop(parseConfig.gamePhaseDetails[0].keyItemDetails[-2].coords.box())
                display(scoreRightIm)
                display(imageTransformation(scoreRightIm, KeyItem.SCORE))
                print(store.lookupImageValue(imageTransformation(scoreRightIm, KeyItem.SCORE), debug=True)               )
            display(result.parsedFrames[idx].image)
            display(result.parsedFrames[idx].image.crop(parseConfig.gamePhaseDetails[0].keyItemDetails[-1].coords.box()))  
            display(result.parsedFrames[idx].image.crop(parseConfig.gamePhaseDetails[0].keyItemDetails[-2].coords.box()))  
        prevscoreLeft = scoreLeft
        prevscoreRight = scoreRight
        prevTime = time
    

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

for game,keyTimes in result.keyTimes.items():
    inGame = keyTimes[GamePhase.IN_GAME]
    times = [240-result.parsedFrames[idx].getValue(KeyItem.TIME, GameSide.NONE).total_seconds() for idx in inGame]
    scoresLeft = [result.parsedFrames[idx].getValue(KeyItem.SCORE, GameSide.LEFT) for idx in inGame]
    scoresRight = [result.parsedFrames[idx].getValue(KeyItem.SCORE, GameSide.RIGHT) for idx in inGame]
    print(inGame)
    print(scoresLeft)
    print(scoresRight)
    left = pd.DataFrame.from_dict({'time':times, 'score':scoresLeft})
    left['side'] = 'left'
    right = pd.DataFrame.from_dict({'time':times, 'score':scoresRight})
    right['side'] = 'right'
    df = pd.concat([left, right])
    sns.lineplot(data=df, x='time', y='score', hue='side')
    plt.show()


In [ ]:
for game, keyTime in result.keyTimes.items():
    print(game)
    if GamePhase.GOAL_SCORED_LEFT_HAND_SIDE in keyTime:
        print(GamePhase.GOAL_SCORED_LEFT_HAND_SIDE)
        for idx in keyTime[GamePhase.GOAL_SCORED_LEFT_HAND_SIDE]:
            print(f'\tTime:',':'.join(str(result.parsedFrames[idx].getValue(KeyItem.TIME, GameSide.NONE)).split(':')[-2:]))
    if GamePhase.GOAL_SCORED_RIGHT_HAND_SIDE in keyTime:
        print(GamePhase.GOAL_SCORED_RIGHT_HAND_SIDE)
        for idx in keyTime[GamePhase.GOAL_SCORED_RIGHT_HAND_SIDE]:
            print(f'\tTime:',':'.join(str(result.parsedFrames[idx].getValue(KeyItem.TIME, GameSide.NONE)).split(':')[-2:]))

In [ ]:
from src.parsing import imageTransformation
from PIL import Image
transformed = imageTransformation(Image.open('../test.jpg'), KeyItem.SCORE)
display(transformed)
#display(store._getHash(transformed))
print(store.lookupImageValue(transformed, debug=True)               )
